In [2]:
import numpy as np
import pandas as pd

In [3]:
# Extract and clean the data frame 

df_data = pd.read_csv('Toronto Dataset.csv')
df_data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
